In [1]:
! export OE_LICENSE=/Users/wangy1/Downloads/oe_license.txt

In [2]:
import torch
import dgl
import espaloma as esp
import math
from openeye import oechem, oedepict, oegrapheme

Using backend: pytorch
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [2]:
g = esp.Graph("[H][C@@](C(=O)N([H])[H])([C@@]([H])(C([H])([H])[H])O[H])N([H])C(=O)C([H])([H])N([H])C(=O)C([H])([H])[H]")

/Users/wangy1/anaconda3/envs/esp/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [1]:
net = torch.load("espaloma-0.2.2.pt")

NameError: name 'torch' is not defined

In [5]:
piptide = esp.Graph("CC(=O)NCC(=O)N[C@@H](C(S))C(=O)N[C@@H](C(C(C)(C)))C(=O)NC")
ligand = esp.Graph("C=CC(=O)N1CCCC(n2nc(-c3ccc(Oc4ccccc4)cc3)c3c(N)ncnc32)C1")
complex = esp.Graph("CC(=O)NCC(=O)N[C@@H](C(S(CCC(=O)N1CCCC(n2nc(-c3ccc(Oc4ccccc4)cc3)c3c(N)ncnc32)C1)))C(=O)N[C@@H](C(C(C)(C)))C(=O)NC")

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 8, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 9, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 42, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 20, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 19, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 21,

In [6]:
net(piptide.heterograph)
net(ligand.heterograph)
net(complex.heterograph)

/Users/wangy1/anaconda3/envs/esp/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: dgl.to_homo is deprecated. Please use dgl.to_homogeneous
  return warnings.warn(message, category=category, stacklevel=1)


Graph(num_nodes={'g': 1, 'n1': 106, 'n2': 220, 'n3': 388, 'n4': 552, 'n4_improper': 174, 'nonbonded': 9438, 'onefour': 552},
      num_edges={('g', 'g_has_n1', 'n1'): 106, ('g', 'g_has_n2', 'n2'): 220, ('g', 'g_has_n3', 'n3'): 388, ('g', 'g_has_n4', 'n4'): 552, ('g', 'g_has_n4_improper', 'n4_improper'): 174, ('g', 'g_has_nonbonded', 'nonbonded'): 9438, ('g', 'g_has_onefour', 'onefour'): 552, ('n1', 'n1_as_0_in_n2', 'n2'): 220, ('n1', 'n1_as_0_in_n3', 'n3'): 388, ('n1', 'n1_as_0_in_n4', 'n4'): 552, ('n1', 'n1_as_0_in_n4_improper', 'n4_improper'): 174, ('n1', 'n1_as_0_in_nonbonded', 'nonbonded'): 9438, ('n1', 'n1_as_0_in_onefour', 'onefour'): 552, ('n1', 'n1_as_1_in_n2', 'n2'): 220, ('n1', 'n1_as_1_in_n3', 'n3'): 388, ('n1', 'n1_as_1_in_n4', 'n4'): 552, ('n1', 'n1_as_1_in_n4_improper', 'n4_improper'): 174, ('n1', 'n1_as_1_in_nonbonded', 'nonbonded'): 9438, ('n1', 'n1_as_1_in_onefour', 'onefour'): 552, ('n1', 'n1_as_2_in_n3', 'n3'): 388, ('n1', 'n1_as_2_in_n4', 'n4'): 552, ('n1', 'n1_as_2

In [7]:
piptide.nodes['g']

NodeSpace(data={'sum_q': tensor([[0.]]), 'sum_s_inv': tensor([[-661.0970]], grad_fn=<GSpMMBackward>), 'sum_e_s_inv': tensor([[38.2593]], grad_fn=<GSpMMBackward>)})

In [8]:
class AtomPartialChargeArcFxn(oegrapheme.OESurfaceArcFxnBase):
    def __init__(self, colorg, g):
        oegrapheme.OESurfaceArcFxnBase.__init__(self)
        self.colorg = colorg
        self.g = g

    def __call__(self, image, arc):
        adisp = arc.GetAtomDisplay()
        if adisp is None or not adisp.IsVisible():
            return False

        atom = adisp.GetAtom()
        if atom is None:
            return False

        idx = atom.GetIdx()
        charge = self.g.nodes['n1'].data['q_hat'][idx].item()
        if charge == 0.0:
            return True
        color = self.colorg.GetColorAt(charge)

        pen = oedepict.OEPen()
        pen.SetForeColor(color)
        pen.SetLineWidth(2.0)

        center = arc.GetCenter()
        radius = arc.GetRadius()
        bAngle = arc.GetBgnAngle()
        eAngle = arc.GetEndAngle()

        edgeAngle = 5.0
        dir = oegrapheme.OEPatternDirection_Outside
        patternAngle = 10.0
        oegrapheme.OEDrawBrickRoadSurfaceArc(image, center, bAngle, eAngle, radius, pen,
                                             edgeAngle, dir, patternAngle)
        return True

    def CreateCopy(self):
        return AtomPartialChargeArcFxn(self.colorg, self.g).__disown__()

In [9]:
class GetAtomLabel(oedepict.OEDisplayAtomPropBase):
    def __init__(self, g):
        oedepict.OEDisplayAtomPropBase.__init__(self)
        self.g = g

    def __call__(self, atom):
        idx = atom.GetIdx()
        charge = self.g.nodes['n1'].data['q_hat'][idx].item()
        return "%.2f" % charge

    def CreateCopy(self):
        copy = GetAtomLabel(g=self.g)
        return copy.__disown__()

In [10]:
class GetBondEq(oedepict.OEDisplayBondPropBase):
    def __init__(self, g):
        oedepict.OEDisplayBondPropBase.__init__(self)
        self.g = g

    def __call__(self, bond):
        bond_idx = bond.GetIdx()
        eq = 0.529177 * g.heterograph.nodes['n2'].data['eq'].flatten()[bond_idx]
        return "%.2f" % eq

    def CreateCopy(self):
        copy = GetBondEq(g=self.g)
        return copy.__disown__()

In [11]:
for name in ["piptide", "ligand", "complex"]:
    g = vars()[name]
    net(g.heterograph)
    opts = oedepict.OE2DMolDisplayOptions(1000, 1000, oedepict.OEScale_AutoScale)
    mol = g.mol.to_openeye()
    oechem.OEMMFFAtomTypes(mol)
    oechem.OEMMFF94PartialCharges(mol)
    oedepict.OEPrepareDepiction(mol)
    oechem.OETriposAtomTypeNames(mol)
    opts.SetAtomPropertyFunctor(GetAtomLabel(g=g))
    # opts.SetBondPropertyFunctor(GetBondEq(g=g))
    # opts.SetBondPropLabelFont(oedepict.OEFont(oechem.OEDarkBlue))
    disp = oedepict.OE2DMolDisplay(mol, opts)
    coloranion = oechem.OEColorStop(-1.0, oechem.OEColor(oechem.OEDarkRed))
    colorcation = oechem.OEColorStop(+1.0, oechem.OEColor(oechem.OEDarkBlue))
    colorg = oechem.OELinearColorGradient(coloranion, colorcation)
    colorg.AddStop(oechem.OEColorStop(0.0, oechem.OEColor(oechem.OEWhite)))

    arcfxn = AtomPartialChargeArcFxn(colorg, g=g)

    for atom in mol.GetAtoms():
        oegrapheme.OESetSurfaceArcFxn(mol, atom, arcfxn)
    oegrapheme.OEDraw2DSurface(disp)
    oedepict.OERenderMolecule("%s_charge.png" % name, disp)

In [12]:
for name in ["piptide", "ligand", "complex"]:
    g = vars()[name]
    net(g.heterograph)
    opts = oedepict.OE2DMolDisplayOptions(1000, 1000, oedepict.OEScale_AutoScale)
    mol = g.mol.to_openeye()
    oechem.OEMMFFAtomTypes(mol)
    oechem.OEMMFF94PartialCharges(mol)
    oedepict.OEPrepareDepiction(mol)
    oechem.OETriposAtomTypeNames(mol)
    # opts.SetAtomPropertyFunctor(GetAtomLabel(g=g))
    opts.SetBondPropertyFunctor(GetBondEq(g=g))
    opts.SetBondPropLabelFont(oedepict.OEFont(oechem.OEDarkBlue))
    disp = oedepict.OE2DMolDisplay(mol, opts)
    coloranion = oechem.OEColorStop(-1.0, oechem.OEColor(oechem.OEDarkRed))
    colorcation = oechem.OEColorStop(+1.0, oechem.OEColor(oechem.OEDarkBlue))
    colorg = oechem.OELinearColorGradient(coloranion, colorcation)
    colorg.AddStop(oechem.OEColorStop(0.0, oechem.OEColor(oechem.OEWhite)))

    #     arcfxn = AtomPartialChargeArcFxn(colorg, g=g)

    #     for atom in mol.GetAtoms():
    #         oegrapheme.OESetSurfaceArcFxn(mol, atom, arcfxn)
    #     oegrapheme.OEDraw2DSurface(disp)
    oedepict.OERenderMolecule("%s_bond.png" % name, disp)

In [ ]:
from openeye import oechem, oedepict
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, "CCC")
oedepict.OEPrepareDepiction(mol)
opts = oedepict.OE2DMolDisplayOptions(30, 30, oedepict.OEScale_AutoScale)
disp = oedepict.OE2DMolDisplay(mol, opts)
oedepict.OERenderMolecule("test.png", disp)